In [1]:
import pandas as pd
import numpy as np
import string

In [2]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from gensim.models import Word2Vec
import os
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [4]:
df = pd.read_csv('../data/final_df.csv')

In [27]:
df.head()

,id,title,tags,embedding
0,1168190,The Wrecking Crew,estranged halfbrothers jonny james reunite fat...,"[-0.28410003, 0.4134046, 0.41264272, 0.1061772..."
1,840464,Greenland 2: Migration,found safety greenland bunker comet clarke dec...,"[-0.23884879, 0.44452164, 0.5361195, 0.0843892..."
2,1084242,Zootopia 2,cracking biggest case zootopias history rookie...,"[-0.18749674, 0.36910042, 0.39568645, 0.051393..."
3,1419406,The Shadow's Edge,macau police brings tracking expert police off...,"[-0.30924657, 0.45583153, 0.46935248, 0.105482..."
4,1368166,The Housemaid,trying escape past millie calloway accepts job...,"[-0.22168699, 0.3902875, 0.41297212, 0.0597435..."


## Text Preprocessing

In [6]:
df.loc[:,'tags']=df['tags'].str.lower()

In [7]:
def remove_punc(text):
    for char in string.punctuation:
        text = text.replace(char,'')
    return text

In [8]:
df.loc[:,'tags']=df['tags'].apply(remove_punc)

In [9]:
def remove_stopwords(text):
    new_text = []
    text_list = text.split()
    english_stopwords = stopwords.words('english')
    for word in text_list:
        if word in english_stopwords:
            pass
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [10]:
df.loc[:,'tags']=df['tags'].apply(remove_stopwords)

In [11]:
df['tags'][1]

'found safety greenland bunker comet clarke decimated earth garrity family must risk everything embark perilous journey across wasteland europe find new home adventure thriller science fiction unitedkingdom unitedstatesofamerica gerardbutler morenabaccarin romangriffindavis tommieearljenkins naturaldisaster shelter survival disaster frantic'

In [12]:
# ps = PorterStemmer()
# def stem_words(text):
#     y=[]
#     text_list = text.split()
#     for i in text_list:
#         y.append(ps.stem(i))
#     return " ".join(y)

In [13]:
# df.loc[:,'tags']=df['tags'].apply(stem_words)

In [14]:
# df['tags'][1]

## Vectorizing Data

In [15]:
movies_descriptions = df['tags'].apply(lambda x: x.split()).tolist()

In [16]:
len(movies_descriptions)

7679

### TF-IDF

In [51]:
vectorizer = TfidfVectorizer(max_features=5000,ngram_range=(1,2),min_df=2)

tfidf_matrix = vectorizer.fit_transform(df['tags'])

In [52]:
similarity_matrix = cosine_similarity(tfidf_matrix)

In [53]:
def recommend(index, top_n=15):
    similarity_scores = list(enumerate(similarity_matrix[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:top_n+1]
    
    recommended_indices = [i[0] for i in similarity_scores]
    
    return df.iloc[recommended_indices]

In [54]:
recommend(98)

,id,title,tags,embedding
129,634649,Spider-Man: No Way Home,peter parker unmasked longer able separate nor...,"[-0.17110406, 0.35963607, 0.4498014, 0.0470330..."
234,324857,Spider-Man: Into the Spider-Verse,struggling find place world juggling school fa...,"[-0.15292275, 0.36186936, 0.454399, 0.04079259..."
399,969681,Spider-Man: Brand New Day,fourth installment spiderman franchise part ph...,"[-0.25007278, 0.35127214, 0.47112763, 0.115089..."
202,557,Spider-Man,bitten genetically altered spider oscorp nerdy...,"[-0.15126576, 0.30750272, 0.39811635, 0.041374..."
383,9806,The Incredibles,bob parr given superhero days log time insuran...,"[-0.16379175, 0.4134027, 0.47503728, 0.0347927..."
858,38055,Megamind,megamind highly intelligent alien supervillain...,"[-0.17912672, 0.38485634, 0.45572072, 0.048112..."
293,559,Spider-Man 3,seemingly invincible spiderman goes allnew cro...,"[-0.16114566, 0.2958022, 0.3704586, 0.05421500..."
316,447365,Guardians of the Galaxy Vol. 3,peter quill still reeling loss gamora must ral...,"[-0.15446325, 0.35353428, 0.45392618, 0.036860..."
190,315635,Spider-Man: Homecoming,following events captain america civil war pet...,"[-0.1858817, 0.42338625, 0.50100607, 0.0466283..."
475,102382,The Amazing Spider-Man 2,peter parker life busy taking bad guys spiderm...,"[-0.09399156, 0.37245002, 0.46704397, -0.00703..."


### Word2Vec

In [46]:
# model = Word2Vec(movies_description,vector_size=100,window =5,min_count = 2,workers=4)

NameError: name 'movies_description' is not defined

In [20]:
# def get_doc_vector(text):
#     words = text.split()
#     vectors = [model.wv[word] for word in words if word in model.wv]
    
#     if len(vectors) == 0:
#         return np.zeros(100)
    
#     return np.mean(vectors, axis=0)

# df['embedding'] = df['tags'].apply(get_doc_vector)

In [22]:
# from sklearn.metrics.pairwise import cosine_similarity

# embeddings = np.vstack(df['embedding'].values)

# similarity_matrix = cosine_similarity(embeddings)

In [36]:
# def recommend(index, top_n=10):
#     similarity_scores = list(enumerate(similarity_matrix[index]))
#     similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
#     # skip itself
#     similarity_scores = similarity_scores[1:top_n+1]
    
#     recommended_indices = [i[0] for i in similarity_scores]
    
#     return df.iloc[recommended_indices]

In [47]:
# df.iloc[98]

id                                                      569094
title                      Spider-Man: Across the Spider-Verse
tags         reuniting gwen stacy brooklyn’s fulltime frien...
embedding    [-0.15665843, 0.3121161, 0.36757156, 0.0494872...
Name: 98, dtype: object

In [37]:
# recommend(98)

,id,title,tags,embedding
10,83533,Avatar: Fire and Ash,wake devastating war rda loss eldest son jake ...,"[-0.1854885, 0.37742049, 0.44902024, 0.0615062..."
7046,14749,Space Buddies,bdawg mudbud budderball rest buddies back time...,"[-0.18307894, 0.38155442, 0.45168895, 0.055787..."
3333,843269,Eureka: Eureka Seven Hi-Evolution,virtual world scab coral corallike information...,"[-0.18772633, 0.3783892, 0.45004898, 0.0619113..."
132,1116465,A Legend,archeologist noticed texture relics discovered...,"[-0.15554646, 0.30009052, 0.35580975, 0.049029..."
879,1274214,Papa Zola: The Movie,papa zola schoolteacher gifted daughter pipi g...,"[-0.20544356, 0.43464494, 0.52955604, 0.062649..."
2744,1267,Meet the Robinsons,lewis brilliant young inventor keen creating t...,"[-0.23307773, 0.46050906, 0.5373262, 0.0843414..."
1508,450465,Glass,series escalating encounters former security g...,"[-0.1839553, 0.3342338, 0.39774436, 0.06080014..."
4523,1067341,Warchief,band guardians tasked protecting messenger mus...,"[-0.19496445, 0.38448384, 0.44491598, 0.057902..."
754,550988,Free Guy,bank teller discovers actually background play...,"[-0.18773615, 0.3864719, 0.4761094, 0.06535667..."
2409,360778,Female Ninjas Magic Chronicles 4: Rebel Forces...,female ninjas stop alliance carrying dastardly...,"[-0.16257408, 0.30438754, 0.35431558, 0.047633..."
